In [10]:
!pip install PyPDF2
!pip install langdetect
!pip install pdfminer

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [11]:
import bs4
import requests
import PyPDF2
import random

# URL of the page to be scraped
lauree_link = "https://amslaurea.unibo.it/view/year/{}.default.html"

# Get the page
page = requests.get(lauree_link.format(2019))
assert page.status_code == 200



In [12]:
import langdetect

soup = bs4.BeautifulSoup(page.content, 'html.parser')
roi = soup.find('div', {"class":"ep_view_page_view_year"})

thesis = roi.find_all('p', recursive=False)

for thesis_text in thesis:
    lang = langdetect.detect_langs(thesis_text.a.text)
    print(lang, thesis_text.a.text)



[it:0.99999618460573] Aspetti tecnici ed economici della cattura e stoccaggio dell’anidride carbonica e potenziali effetti sul cambiamento climatico.
[en:0.9999941285398783] Fabrication and characterization of polyetherimide Supported Ionic Liquid Membranes for gas separation.
[it:0.999997551517533] L'ottimizzazione del layout del cantiere edile attraverso l'utilizzo di algoritmi genetici multi-obiettivo: caso di studio.
[it:0.9999966765233804] Studio e sviluppo di descrittori locali per nuvole di punti basati su proprietà geometriche.
[it:0.9999968452537227] Anelli fattoriali: aspetti algebrici e geometrici.
[it:0.9999960804211367] Caratterizzazione chimica ed elettrica di polietilene reticolato sottoposto ad invecchiamento termico e radiativo.
[en:0.9999966481607026] Deep reinforcement learning optimization of video streaming.
[it:0.9999962323731227] Sviluppo concettuale, implementazione e realizzazione prototipale di macchina per misure millesimali.
[it:0.9999960704488169] ottimizza

In [13]:

def filter_by_lang(thesis_list, target_lang = 'it'):
    """
    input: il a list of tag with name, title, tipo laurea, uni, corso
    output: a list of link to the thesis (with available text)
    """
    return list(filter(
        lambda title: langdetect.detect(title.a.text) == target_lang,
        thesis_list
    ))

def get_page_with_thesis(link):
    """
    input: link to the page with the thesis
    output: a list of tag with name, title, tipo laurea, uni, corso
    """
    page = requests.get(link)
    assert page.status_code == 200
    soup = bs4.BeautifulSoup(page.content, 'html.parser')
    roi = soup.find('div', {"class":"ep_view_page_view_year"})
    return roi.find_all('p', recursive=False)

def analyze_link(thesis):
    """
    input: il a list of tag with name, title, tipo laurea, uni, corso
    output: a list of link to the thesis (with available text)
    """
    no_full_text = lambda testo : "Documento full-text non disponibile" in testo
    full_text_reserved = lambda testo : "Documento ad accesso riservato" in testo

    return list(map(
        lambda ls: ls.find('a')['href'],
        filter(
            lambda title: langdetect.detect(title.a.text) == 'it' and not no_full_text(title.text) and not full_text_reserved(title.text),
            thesis
        )
    ))



In [14]:

thesis_links = []

for y in range(2012,2024):
    # print(lauree_link.format(y))
    list_thesis = get_page_with_thesis(lauree_link.format(y))
    # print(1, len(list_thesis))
    list_thesis = filter_by_lang(list_thesis)
    # print(2, len(list_thesis))
    list_thesis = analyze_link(list_thesis)
    print(y, len(list_thesis))

    thesis_links.extend(list_thesis)

2012 589
2013 607
2014 696
2015 693
2016 676
2017 509
2018 539
2019 588
2020 471
2021 522
2022 428
2023 109


In [15]:
thesis_links = list(set(thesis_links))
print(len(thesis_links))

6427


In [16]:
import io
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams

def pdf_to_text(raw_pdf, from_page = 6, exclude_page = 4):
    # PDFMiner boilerplate
    rsrcmgr = PDFResourceManager()
    sio = io.StringIO()
    device = TextConverter(rsrcmgr, sio, laparams=LAParams())
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    # get text from file

    pdf_pages = list(PDFPage.get_pages(io.BytesIO(raw_pdf)))[from_page:-exclude_page]

    for page in pdf_pages:
        interpreter.process_page(page)

    # Get text from StringIO
    text = sio.getvalue()
    # close objects
    device.close()
    sio.close()

    return text

def get_pdf_text2(link, from_page = 4, exclude_page = 3):
    raw_pdf = requests.get(link).content
    pdf = PyPDF2.PdfReader(io.BytesIO(raw_pdf))
    text = "".join(
        page.extract_text()
        for page in pdf.pages[from_page:-exclude_page]
    )
    return text


def get_pdf_text(link, from_page = 4, exclude_page = 3) -> str:
    raw_pdf = requests.get(link).content
    return pdf_to_text(raw_pdf, from_page, exclude_page)

def analyze_thesis_page(link):
    """
    get to return title, pdf link, is actually italian (from abstract) 
    """
    page = requests.get(link)
    assert page.status_code == 200
    soup = bs4.BeautifulSoup(page.content, 'html.parser')
    title = soup.find('h1', {"id":"mail_title"})
    title = title.text.lower().replace("\n", "")
    title = title.replace(' ', '_').replace(".", "")

    pdf_link = soup.find("a", {"class":"ep_document_link"})
    pdf_link = pdf_link['href']

    abstract = soup.find('p', {"id":"abstract_body"})
    abstract_lang = langdetect.detect(abstract.text)

    return title, pdf_link, abstract_lang == 'it'

CHAR_CONVERSION_DICT = {
    "-\n": "",
    "´e": "é",
    "`e": "è",
    "`u": "ù",
    "`a": "à",
    "`o": "ò",
    "`i": "ì",
    "`E": "È",
    "ﬀ": "ff",
    "ﬁ": "fi",
    "ﬂ": "fl",
    "ﬃ": "ffi",
    "ﬄ": "ffl",
    '–': '-',
    '—': '-',
    '”': '"',
    '“': '"',
    '−': '-',
    '’': "'",
    '…': '...',
    '⋯': '...',        
    '‘': "'",
    "’": "'",
    '∗': '*',
    '  ': ' ',
    '   ': ' ',
    '': '•',
    '•': '-',
    '\t': ' ',
    '\r': ' ',
    '𝟐': '2',
    '𝟑': '3',
    '𝐷': 'D',
    '𝑆': 'S',
    '𝑃': 'P',
    '𝑑': 'd',
    '♯': '#',
    '𝑤': 'w',
    '': ' ',
    'ℳ': 'M',
    '𝑞': 'q',
    '𝑛': 'n',
    '𝑉': 'V',
    '𝒜': 'A',
    'ƒ': 'f',
'∪': 'U',
'⁄': '/',
'ˆ': '^',
'ä' : 'a',
'‐' : '-',
'': ' ',
'⇔': '↔',
'º': '°',
'∼': '~',
'(cid:18)a': 'à',
u'\uF8FA': ' ',
u"\uF0E0": ' ',
    chr(28): ' ',
}

In [17]:
"""
here is the plan to test the way to handle special char

get a rappresentative sample of thesis
1) get 60 random thesis
2) create a test folder
3) request the pdf
4) convert the pdf to text
5) create a Counter for every thesis
6) save the text in the test folder

estimate the char to cut off
1) merge the Counters 
2) identify the least common chars (with a threshold, like 0.2% of the total)
3) save the char in a list

create a function to remove the char
1) reload the text
2) remove the char

"""

'\nhere is the plan to test the way to handle special char\n\nget a rappresentative sample of thesis\n1) get 60 random thesis\n2) create a test folder\n3) request the pdf\n4) convert the pdf to text\n5) create a Counter for every thesis\n6) save the text in the test folder\n\nestimate the char to cut off\n1) merge the Counters \n2) identify the least common chars (with a threshold, like 0.2% of the total)\n3) save the char in a list\n\ncreate a function to remove the char\n1) reload the text\n2) remove the char\n\n'

In [18]:
from collections import Counter
import random
import os

N_THESIS = 1300
THESIS_FOLDER = "test_thesis"
COUNTERS = []
os.makedirs(THESIS_FOLDER, exist_ok=True)
random_thesis = random.sample(thesis_links, N_THESIS)
for i, l in enumerate(random_thesis):
    try:
        title, pdf_link, is_it = analyze_thesis_page(l)
        raw_pdf = requests.get(pdf_link).content
        extr = pdf_to_text(raw_pdf)
    except:
        print("error", l)
        continue
    if not is_it:
        print("not it", l)
        continue


    for to_remove, to_replace in CHAR_CONVERSION_DICT.items():
        extr = extr.replace(to_remove, to_replace)

    # also remove strange chars
    for c in extr:
        if len(c.encode("utf-8")) >= 3:
            extr = extr.replace(c, " ")
    
    COUNTERS.append(
        Counter(list(extr))
    )

    with open(os.path.join(THESIS_FOLDER, f"{title}.txt"), "w+") as f:
        f.write(extr)




error http://amslaurea.unibo.it/4556/
error http://amslaurea.unibo.it/3548/
error http://amslaurea.unibo.it/4571/
error http://amslaurea.unibo.it/3442/
error http://amslaurea.unibo.it/5532/
error http://amslaurea.unibo.it/4668/
not it http://amslaurea.unibo.it/13417/
error http://amslaurea.unibo.it/4623/
not it http://amslaurea.unibo.it/18166/
error http://amslaurea.unibo.it/3270/
not it http://amslaurea.unibo.it/16218/
error http://amslaurea.unibo.it/4684/


FileNotFoundError: [Errno 2] No such file or directory: 'test_thesis/deidrofluorurazione_di_copolimeri_di_vdf/hfp_in_presenza_di_alcali.txt'

In [ ]:
# merge counters

merged = COUNTERS[0]
for c in COUNTERS[1:]:
    merged += c

# print least common
print(merged.most_common()[-10:])
# print most common
print(merged.most_common()[:10])
print(len(merged))

[('½', 1), ('²', 1), ('§', 1), ('ñ', 1), ('˛', 1), ('ã', 1), ('˚', 1), ('Á', 1), ('ć', 1), ('¸', 1)]
[(' ', 305046), ('i', 180748), ('e', 180658), ('a', 147328), ('o', 133406), ('t', 111830), ('n', 109615), ('r', 94393), ('l', 89543), ('\n', 80041)]
238


In [ ]:
for character in merged.most_common()[-60:-30]:
    print(character[0], character[1] , len(character[0].encode('utf-8')))

× 5 2
ç 5 2
ö 5 2
✓ 5 3
̂ 4 2
⇒ 4 3
® 4 2
← 4 3
€ 4 3
Δ 4 2
≈ 4 3
μ 4 2
š 4 2
∏ 4 3
„ 3 3
∫ 3 3
∼ 3 3
 3 3
 3 3
ɛ 3 2
ƍ 3 2
â 3 2
ø 3 2
ó 3 2
∝ 3 3
÷ 2 2
º 2 2
ᴰ 2 3
ˢ 2 2
ʎ 2 2
